# Wine Quality Classification

The notebook is intended to develop & validate a model for multi-class classification of the Wine Quality.

In [ ]:
# Import Standard Libraries
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import zscore

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
# Define Seaborn theme parameters
theme_parameters =  {
    'axes.spines.right': False,
    'axes.spines.top': False,
    'grid.alpha':0.3,
    'figure.figsize': (16, 6),
    'font.family': 'Andale Mono',
    'axes.titlesize': 24,
    'figure.facecolor': '#E5E8E8',
    'axes.facecolor': '#E5E8E8'
}

# Set the theme
sns.set_theme(style='whitegrid',
              palette=sns.color_palette('deep'), 
              rc=theme_parameters)

In [ ]:
# Notebook's variables
train_data_path = os.path.join('./../../data/S3E5/wine_quality_train.csv')
test_data_path = os.path.join('./../../data/S3E5/wine_quality_test.csv')

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

# Exploratory Data Analysis

## Train Feature & Label Distribution

In [ ]:
# Plot the histograms of each feature
figure, ax = plt.subplots(3, 4, figsize=(16, 9))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' column)
for index, column_name in enumerate(train_data.columns[1:]):
    
    # Plot data
    sns.histplot(data=train_data[column_name], 
                 ax=ax[index])
    
    ax[index].set_title(column_name, 
                        fontsize=14, 
                        fontweight='bold')
    
    ax[index].tick_params(labelrotation=45)
    
plt.suptitle('Feature & Label Distrubtion', 
             fontsize=20)
    
plt.tight_layout()

The following features have a skewed distribution:
- `fixed acidity`
- `citric acid`
- `residual sugar`
- `free sulfur dioxide`
- `total sulfur dioxide`
- `sulphates`
- `alcohol`

It would be useful to use the Z-Score Outliers Filter.
<br>

In addition, it is possible to see that the label classes 3, 4 and 8 do not have a lots of records. That is an imbalanced data problem. Consider to use a Stratified K-Fold during the training of the model.

## Train Feature Distribution per Label

In [ ]:
# Plot the box plot of each feature per label
figure, ax = plt.subplots(3, 4, figsize=(16, 12))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' column and 'quality' column)
for index, column_name in enumerate(train_data.columns[1:-1]):
    
    # Plot data
    sns.boxplot(data=train_data,
                x='quality',
                y=column_name,
                ax=ax[index])
    
        
# Remove the empty subplot
figure.delaxes(ax[-1])

# Set title plot
plt.suptitle('Feature Distrubtion per Label', 
             fontsize=20, 
             fontweight='bold')

plt.tight_layout()

plt.show()

There is a positive non-linear relationship between the following features and the `Quality`:
- `sulphates`
- `alcohol`

Thre is a negative non-linear relationship between the following features the the `Quality`:
- `voltatile acidity`
- `density`

## Pearson Correlation

In [ ]:
# Compute the correlation matrix
correlation_matrix = train_data.iloc[:, 1:].corr()

In [ ]:
# Generate a mask for the upper triangle
correlation_mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

In [ ]:
# Define figure and axis
figure, ax = plt.subplots(figsize=(12, 8))

# Plot the correlation matrix
sns.heatmap(correlation_matrix, 
            mask=correlation_mask, 
            cmap='mako',
            vmax=1.0, 
            vmin=-1.0, 
            center=0, 
            square=True, 
            linewidths=.5, 
            annot=True,
            annot_kws={'fontsize': 8},
            cbar_kws={"shrink":.8, 'orientation':'vertical'})

# Set title
ax.set_title('Pearson Correlation', 
             fontsize=20, 
             fontweight='bold')

plt.tight_layout()

plt.show()

The following features show a significant positive correlation:
- `citric acid` and `fixed acidity`
- `density` and `fixed acidity`
- `total sulfur dioxide` and `free sulfur dioxide`

The following features show a significant negative correlation:
- `citric acid` and `volatile acidity`
- `pH` and `fixed acidity`

## Train vs Test Feature & Label Distribution

In [ ]:
# Plot the KDE of each feature
figure, ax = plt.subplots(3, 4, figsize=(16, 12))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' and 'quality' columns)
for index, column_name in enumerate(train_data.columns[1:-1]):
    
    # Plot data
    sns.kdeplot(data=train_data[column_name],
                label='Train',
                ax=ax[index])
    
    sns.kdeplot(data=test_data[column_name],
                label='Test',
                ax=ax[index])
    
    ax[index].set_title(column_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
    # Retrieve legend information
    handles = ax[index].get_legend_handles_labels()[0]
    labels = ax[index].get_legend_handles_labels()[1]
    ax[index].legend().remove()
    
# Remove the empty subplot
figure.delaxes(ax[-1])

# Set the legend
figure.legend(handles, 
              labels, 
              loc='upper center', 
              bbox_to_anchor=(0.5, 1.03), 
              fontsize=12,
              ncol=2)

plt.tight_layout()

There are no strong differences between the feature distribution of the train set and the test set.

## Count Outliers with the Z-Score across Quality Classes

In [ ]:
# Compute the Z-Score for the feature columns across 'quality' classes
z_scores = train_data.iloc[:, 1:-1].groupby(train_data['quality'], 
                                            group_keys=True).apply(zscore)

In [ ]:
# Consider as an 'outlier' every records with a Z-Score bigger than 2 SDs in absolute value terms
outliers = z_scores.abs().ge(2).groupby(z_scores.index.get_level_values(0)).sum()

In [ ]:
# Plot outlisers per feature across 'quality' classes
figure, ax = plt.subplots(3, 4, figsize=(16, 9))
ax = ax.flatten()

# Fetch the data to plot (exclude the 'id' and 'quality' columns)
for index, column_name in enumerate(outliers.columns):
    
    # Plot data
    sns.barplot(data=outliers,
                x=outliers.index,
                y=column_name,
                ax=ax[index])
    
    ax[index].set_title(column_name, fontsize=14)
    
    ax[index].tick_params(labelrotation=45)
    
# Remove the empty subplot
figure.delaxes(ax[-1])

# Set title plot
plt.suptitle('Outliers Count', 
             fontsize=20, 
             fontweight='bold')
    
plt.tight_layout()

The quality classes 5, 6 and 7 show the highest amount of outliers.

## Train Feature Pairplot

In [ ]:
# Plot the Pairplot between the features
sns.pairplot(train_data.drop(columns=['Id', 'quality']),
             kind="reg",
             diag_kind='kde',
             plot_kws={'line_kws':{'color':'red'}},
             corner=True)

# Set title plot
plt.suptitle('Train Feature Pairplot', 
             fontsize=20, 
             fontweight='bold')

plt.tight_layout()

plt.show()

## Conclusions
- Several numerical features present a right-skew distribution -> Use of a StandardScalre
- `quality` target variable have imbalanced classes -> Use a Stratified K-Fold
- Create a `sulphates for alcohol` feature through `sulphates` * `alcohol`
- Create a `volatile acidity for density` feature through `volatile acidity` * `density`
- Create a `sulphates over density` feature through `sulphates` / `density`
- Create a `alcohol over density` feature through `alcohol` / `density`
- Create a `sulphates over volatile acidity` feature through `sulphates` / `volatile acidity`
- Create a `alcohol over volatile acidity` feature through `alcohol` / `volatile acidity`
- Create a `citric acid for fixed acidity` feature through `citric acid` * `fixed acidity`
- Create a `density for fixed acidity` feature through `density` * `fixed acidity`
- Create a `total sulfur dioxide for free sulfur dioxide` feature through `total sulfur dioxide` * `free sulfur dioxide`
- Create a `citric acid for volatile acidity` feature through `citric acid` * `volatile acidity`
- Create a `pH for fixed acidity` feature through `pH` * `fixed acidity`

# Data Preparation

## Feature Engineering

In [ ]:
def compute_engineered_features(data: pd.DataFrame) -> pd.DataFrame:
    
    """
    Create a pre-defined set of engineered feature to the input DataFrame
    
    Args:
        data Pandas.DataFrame input
    
    Returns;
        data Pandas.DataFrame with additional engineered features
    """
    
    # Create a `sulphates for alcohol` feature through `sulphates` * `alcohol`
    data['sulphates for alcohol'] = round(data['sulphates'] * data['alcohol'], 2)
    
    # Create a `volatile acidity for density` feature through `volatile acidity` * `density`
    data['volatile acidity for density'] = round(data['volatile acidity'] * data['density'], 2)
    
    # Create a `sulphates over density` feature through `sulphates` / `density`
    data['sulphates over density'] = round(data['sulphates'] / data['density'], 2)
    
    # Create a `alcohol over density` feature through `alcohol` / `density`
    data['alcohol over density'] = round(data['alcohol'] / data['density'], 2)
    
    # Create a `sulphates over volatile acidity` feature through `sulphates` / `volatile acidity`
    data['sulphates over volatile acidity'] = round(data['sulphates'] / data['volatile acidity'], 2)
    
    # Create a `alcohol over volatile acidity` feature through `alcohol` / `volatile acidity`
    data['alcohol over volatile acidity'] = round(data['alcohol'] / data['volatile acidity'], 2)
    
    # Create a `citric acid for fixed acidity` feature through `citric acid` * `fixed acidity`
    data['citric acid for fixed acidity'] = round(data['citric acid'] * data['fixed acidity'], 2)
    
    # Create a `density for fixed acidity` feature through `density` * `fixed acidity`
    data['density for fixed acidity'] = round(data['density'] * data['fixed acidity'], 2)
    
    # Create a `total sulfur dioxide for free sulfur dioxide` feature through `total sulfur dioxide` * `free sulfur dioxide`
    data['total sulfur dioxide for free sulfur dioxide'] = round(data['total sulfur dioxide'] * data['free sulfur dioxide'], 2)
    
    # Create a `citric acid for volatile acidity` feature through `citric acid` * `volatile acidity`
    data['citric acid for volatile acidity'] = round(data['citric acid'] * data['volatile acidity'], 2)
    
    # Create a `pH for fixed acidity` feature through `pH` * `fixed acidity`
    data['pH for fixed acidity'] = round(data['pH'] * data['fixed acidity'], 2)

In [ ]:
# Apply the feature engineering
compute_engineered_features(train_data)
compute_engineered_features(test_data)

## Features and Labels Definition

In [ ]:
# Define features and labels
numerical_features = train_data.columns[1:12]

numerical_engineered_featuers = train_data.columns[13:]

label = ['quality']

## Numerical Features Preprocessing Pipeline

In [ ]:
# Numerical features pipeline
numerical_features_pipeline = Pipeline(steps=[
    ('numerical_scaler', StandardScaler())
])

## Bundle Data Preprocessing Steps

In [ ]:
# Bunlde data preprocessing steps
data_preprocessor = ColumnTransformer(
    transformers=[
        ('numerical_preprocessing', 
         numerical_features_pipeline, numerical_features + numerical_engineered_featuers),
    ])